In [111]:
from collections import defaultdict
from libpy import readfile

filename = "2024/day16/input/sample.txt"
#filename = "2024/day16/input/sample2.txt"
#filename = "2024/day16/input/sample3.txt"
filename = "2024/day16/input/input.txt"
lines = readfile.lines(filename)

width = len(lines[0])
height = len(lines)

HORIZONTAL = 0
VERTICAL = 1

start = None
end = None
roads = []
crosses_horizontal = []
crosses_vertical = []
for y in range(height):
    for x in range(width):
        if lines[y][x] == "S":
            start = (x, y)
        elif lines[y][x] == "E":
            end = (x, y)
        
        # horizontal roads
        if lines[y][x] != "#" and lines[y][x - 1] == "#" or (x, y) in crosses_horizontal:
            l = 0
            while lines[y][x + l + 1] != "#":
                l += 1
                if lines[y - 1][x + l] != "#" or lines[y + 1][x + l] != "#":
                    roads.append(((x, y), (x + l, y), l, HORIZONTAL))
                    crosses_horizontal.append((x + l, y))
            if l > 0:
                roads.append(((x, y), (x + l, y), l, HORIZONTAL))
        
        # vertical
        if lines[y][x] != "#" and lines[y - 1][x] == "#" or (x, y) in crosses_vertical:
            l = 0
            while lines[y + l + 1][x] != "#":
                l += 1
                if lines[y + l][x - 1] != "#" or lines[y + l][x + 1] != "#":
                    roads.append(((x, y), (x, y + l), l, VERTICAL))
                    crosses_vertical.append((x, y + l))
            if l > 0:
                roads.append(((x, y), (x, y + l), l, VERTICAL))

roads_dict = defaultdict(list)
for road in roads:
    roads_dict[road[0]].append((road[1], road[2], road[3]))
    roads_dict[road[1]].append((road[0], road[2], road[3]))

print(roads_dict)
                

defaultdict(<class 'list'>, {(1, 1): [((5, 1), 4, 0), ((5, 1), 4, 0), ((1, 3), 2, 1), ((1, 7), 6, 1), ((1, 7), 6, 1)], (5, 1): [((1, 1), 4, 0), ((1, 1), 4, 0), ((5, 3), 2, 1)], (1, 3): [((1, 1), 2, 1), ((3, 3), 2, 0), ((3, 3), 2, 0), ((1, 7), 4, 1), ((1, 7), 4, 1)], (1, 7): [((1, 1), 6, 1), ((1, 1), 6, 1), ((1, 3), 4, 1), ((1, 3), 4, 1), ((3, 7), 2, 0), ((3, 7), 2, 0)], (5, 3): [((5, 1), 2, 1)], (7, 1): [((9, 1), 2, 0), ((11, 1), 4, 0), ((11, 1), 4, 0), ((7, 5), 4, 1), ((7, 5), 4, 1)], (9, 1): [((7, 1), 2, 0), ((11, 1), 2, 0), ((11, 1), 2, 0), ((9, 13), 12, 1), ((9, 13), 12, 1)], (11, 1): [((7, 1), 4, 0), ((7, 1), 4, 0), ((9, 1), 2, 0), ((9, 1), 2, 0), ((11, 3), 2, 1), ((11, 3), 2, 1)], (7, 5): [((7, 1), 4, 1), ((7, 1), 4, 1), ((3, 5), 4, 0), ((3, 5), 4, 0)], (9, 13): [((9, 1), 12, 1), ((9, 1), 12, 1), ((13, 13), 4, 0), ((13, 13), 4, 0)], (11, 3): [((11, 1), 2, 1), ((11, 1), 2, 1), ((13, 3), 2, 0), ((13, 3), 2, 0)], (13, 1): [((15, 1), 2, 0), ((15, 1), 2, 0)], (15, 1): [((13, 1), 2, 0)

Part A

In [113]:
weights = dict([(((x[0], x[1]), HORIZONTAL), -1) for x in set(roads_dict)] + [(((x[0], x[1]), VERTICAL), -1) for x in set(roads_dict)])

weights[(start, HORIZONTAL)] = 0
weights[(start, VERTICAL)] = 1000

positions_to_evaluate = set([start])
while len(positions_to_evaluate) > 0:
    new_positions_to_evaluate = set()
    for position in positions_to_evaluate:
        position_weight_h = weights[(position, HORIZONTAL)]
        position_weight_v = weights[(position, VERTICAL)]
        
        for road in roads_dict[position]:
            new_weight = 0
            if road[2] == HORIZONTAL:
                new_weight = position_weight_h + road[1]
                if position_weight_h < position_weight_v + 1000:
                    new_weight = position_weight_v + road[1] + 1000
            else:
                new_weight = position_weight_v + road[1]
                if position_weight_v < position_weight_h + 1000:
                    new_weight = position_weight_h + road[1] + 1000
                    
            dest = weights[(road[0], road[2])] 
            if dest == -1 or dest > new_weight:
                weights[(road[0], road[2])] = new_weight
                new_positions_to_evaluate.add(road[0])
                
    positions_to_evaluate = new_positions_to_evaluate

print(min(weights[(end, HORIZONTAL)], weights[(end, VERTICAL)]))


123540


Part B

In [115]:
weights = dict([(((x[0], x[1]), HORIZONTAL), -1) for x in set(roads_dict)] + [(((x[0], x[1]), VERTICAL), -1) for x in set(roads_dict)])
routeparts = dict([(((x[0], x[1]), HORIZONTAL), set()) for x in set(roads_dict)] + [(((x[0], x[1]), VERTICAL), set()) for x in set(roads_dict)])
    
weights[(start, HORIZONTAL)] = 0
weights[(start, VERTICAL)] = 1000

positions_to_evaluate = set([start])
while len(positions_to_evaluate) > 0:
    new_positions_to_evaluate = set()
    for position in positions_to_evaluate:
        position_weight_h = weights[(position, HORIZONTAL)]
        position_weight_v = weights[(position, VERTICAL)]
        
        for road in roads_dict[position]:
            new_weight = 0
            new_routeparts = None
            if road[2] == HORIZONTAL:
                new_weight = position_weight_h + road[1]
                new_routeparts = routeparts[(position, HORIZONTAL)].copy()
                if (position_weight_v + 1000 < position_weight_h and position_weight_v >= 0) or position_weight_h == -1:
                    new_weight = position_weight_v + road[1] + 1000
                    new_routeparts = routeparts[(position, VERTICAL)].copy()
                elif position_weight_v + 1000 == position_weight_h and position_weight_v >= 0:
                    new_routeparts = set(list(new_routeparts) + list(routeparts[(position, VERTICAL)]))
            else:
                new_weight = position_weight_v + road[1]
                new_routeparts = routeparts[(position, VERTICAL)].copy()
                if (position_weight_h + 1000 < position_weight_v and position_weight_h >= 0) or position_weight_v == -1:
                    new_weight = position_weight_h + road[1] + 1000
                    new_routeparts = routeparts[(position, HORIZONTAL)].copy()
                elif position_weight_h + 1000 < position_weight_v and position_weight_h >= 0:
                    new_routeparts = set(list(new_routeparts) + list(routeparts[(position, HORIZONTAL)]))

            dest = weights[(road[0], road[2])]

            if dest == -1 or dest > new_weight:
                weights[(road[0], road[2])] = new_weight
                routeparts[(road[0], road[2])] = new_routeparts
                if road[2] == HORIZONTAL:
                    for x in range(min(road[0][0], position[0]), max(road[0][0], position[0]) + 1):
                        routeparts[(road[0], road[2])].add((x,road[0][1]))
                else:
                    for y in range(min(road[0][1], position[1]), max(road[0][1], position[1]) + 1):
                        routeparts[(road[0], road[2])].add((road[0][0], y))
                new_positions_to_evaluate.add(road[0])
            elif dest == new_weight:
                old_route = routeparts[(road[0], road[2])]
                routeparts[(road[0], road[2])] = set(list(routeparts[(road[0], road[2])]) + list(new_routeparts))
                if road[2] == HORIZONTAL:
                    for x in range(min(road[0][0], position[0]), max(road[0][0], position[0]) + 1):
                        routeparts[(road[0], road[2])].add((x,road[0][1]))
                else:
                    for y in range(min(road[0][1], position[1]), max(road[0][1], position[1]) + 1):
                        routeparts[(road[0], road[2])].add((road[0][0],y))
                        
                if routeparts[(road[0], road[2])] != old_route:
                    new_positions_to_evaluate.add(road[0])
                                
    positions_to_evaluate = new_positions_to_evaluate

if weights[(end, HORIZONTAL)] < weights[(end, VERTICAL)]:
    print(len(routeparts[(end, HORIZONTAL)]))
else:
    print(len(routeparts[(end, VERTICAL)]))
    


665
